# ETL

In [1]:
#Importamos los paquete necesarios:
import os
import pandas as pd
import glob ##New package learned   
from sqlalchemy import create_engine

## Extract:


In [15]:
def extractANDclean_CSV(file_csv):
    '''Transformamos un CSV en un datafram y lo renombramos como documento procesado
    
    IN: archivo CSV
    OUT: DataFrame, csv_renamed
    '''
    print(f"Processing {file_csv} as data")
    return pd.read_csv(file_csv)

def extractANDclean_JSON(file_json):
    '''Transformamos un JSON en un datafram y lo renombramos como documento procesado

    IN: archivo JSON
    OUT: DataFrame, JSON_renamed
    '''
    print(f"Processing {file_json} as data")
    return pd.read_json(file_json, lines=True) ###REMARK

def extractANDclean_TXT(file_txt):
    '''Transformamos un TXT en un datafram y lo renombramos como documento procesado

    IN: archivo TXT
    OUT: DataFrame, TXT_renamed
    '''
    print(f"Processing {file_txt} as data")
    return pd.read_csv(file_txt,sep='|') ###REMARK

def extractANDclean_XLSX(file_xlsx):
    '''Transformamos un XLSX en un datafram y lo renombramos como documento procesado

    IN: archivo XLSX
    OUT: DataFrame, XLSX_renamed
    '''
    print(f"Processing {file_xlsx} as data")
    return pd.read_excel(file_xlsx)

In [ ]:
#Damos la ruta al directorio de datos "sucios":
dirty_data = 'dirty_data/*'

#Generamos una lista que contendra el total de los DF:
list_of_dfs = []

#Diferenciamos por tipo de documento para poder dirigirlo a la función adecuada:
for doc in glob.glob(dirty_data): ###REMARK
    
    #Generamos los df con las funciones y los añadimos a la lista de df
    if doc[-4:] == ".csv": df_csv = list_of_dfs.append(extractANDclean_CSV(doc))
    elif doc[-5:] == ".json": df_json = list_of_dfs.append(extractANDclean_JSON(doc))
    elif doc[-5:] == ".xlsx": df_xlsx = list_of_dfs.append(extractANDclean_XLSX(doc))
        #Presuponemos que el separador para los TXT será "|":
    elif doc[-4:] == ".txt": df_txt = list_of_dfs.append(extractANDclean_TXT(doc))
    
    #Levantamos un error si el formato no existe:
    else: 
        raise TypeError(f"Documento con formato no contemplado: {doc}")
    

#Concatenamos todos los df con todos los registros de los diferentes documentos:
df_sales= pd.concat(list_of_dfs, ignore_index=True) ###REMARK

Processing dirty_data/dataset_ventas.csv as data
Processing dirty_data/dataset_ventas.xlsx as data
Processing dirty_data/dataset_ventas.txt as data
Processing dirty_data/dataset_ventas.json as data
       id_venta       fecha   cliente producto precio cantidad
0             1  2023.09.26   hicham     Mouse    -10      NaN
1             2  2023.12.27     MARTA  Teclado  ERROR      NaN
2             3  2023-05-05      luis   Cables   1200      5.0
3             4         NaN    PeDro   Teclado  ERROR      5.0
4             5  2023.08.15     sofia    Mouse   1200      2.0
...         ...         ...       ...      ...    ...      ...
39995      9996  2023-07-13    PeDro    Laptop    -10     None
39996      9997  2023.07.29     MARTA   Cables     15        2
39997      9998  2023-11-19      JUAN   Cables    350        5
39998      9999  2023.11.29      JUAN    Mouse   1200        5
39999     10000                 CARLA  Teclado     15        1

[40000 rows x 6 columns]
